# Assignment 1: First List Generation
## Computational Methods in Psychology and Neuroscience
### Psychology 4215/7215 --- Fall 2023

# Objectives

Upon completion of this assignment, the student will have:

1. Created unique trial conditions

2. Randomly generated lists to use in a experiment.


# Assignment

* Write code in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A01_FirstListGen_mst3k).

## Design

Your assignment is to write a script that creates lists of dictionaries that you will later present to participants as part of an experiment.  

The script should be configurable such that you can specify different
numbers of lists and trials, along with other details specific to the
experiment you decide to do.

Each dictionary represents a trial and should contain all the
information necessary to identify the stimulus to be presented,
details about that stimulus, and the condition in which to present it.
This information will be experiment-specific, as outlined below.

This assignment acts as a warm-up list generation that is an extension of the flanker example from class. Your full list generation assignment will come next and you will have a choice of experiment to code.

  
* ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on Canvas.***  

## Perceptual Decision-Making with Interrogation and Confidence

The main question of this study is whether we see changes in calibration between accuracy and confidence for perceptual decisions under time pressure. In prior collaborative work, my lab has demonstrated that taking away evidence after short periods of time can affect the decision-making process (see Palestro et al., 2018; https://compmem.org/assets/pdf/Palestro.etal.2018b.pdf). Here we will also probe participant confidence in their decisions after varying levels amounts of time to assess a stimulus for evidence guiding the choice, but that will not affect the list generation.

We will present random moving dot stimuli to participants at various levels of coherence. The task is to indicate the direction (left or right) that most of the dots are moving. I'll show examples in class, but for now you simply need to know that the higher the coherence of the dot movement, the easier the trial. The more time a participant has to evaluate the movement also gives rise to better performance. 

Thus, we will be crossing two factors in this experiment, coherence and interrogation time, but we also need to control for the direction of the coherent movement so that both left and right are equally represented for each condition.

You task is to write list-generation code that crosses:

- Coherence levels of 0.0, 0.1, 0.2, 0.3
  - Note, these should be fully crossed, so you should have trials like:
    - 0.0, 0.1
    - 0.1, 0.0
    - 0.0, 0.2
    - 0.2, 0.0
    - ...
- Interrogation times of .200, .300, .500, .800 seconds

Each trial will be defined by a dictionary, such as:

`{'left_coh': 0.2, 'right_coh': .1, 'inter_time': 500}`

Your complete code should allow for specification of the number of repetitions of all conditions in a single list, as well as how many lists you'd like to run, giving rise to a list of lists of dictionaries that you write out to a pickle file.

***Note, You can use the flanker example from class as a guide!***


# My work

## Requirements

Generation
- Coherence levels
    - These should be full crossed
- Interval time
    - This should be crossed with the coherence levels
- order effects
    - Not sure we care about this one


Final output
- Final output
    - One large list
    - One list inside for each subject/block
    - each subject/block's list is a list of dictionaries with the three params
        - `{'left_coh': 0.2, 'right_coh': .1, 'inter_time': 500}`

## Coding the list gen

In [1]:
import numpy as np  # for classic numpy array magic
import random       # for randomizing list order
import pickle       # for saving out lists

### Create a test function

In [2]:
def list_gen_test(*args):
    list_generated = gen_pdm_list(args[0], args[1], args[2], args[3])
    assert len(list_generated) == args[3], "blocks not correct" 

    # calculate the theoretical number of trials (coherence**2 * interogatoins_times * number of reps per block)
    correct_trial_num = (len(args[0])**2) * len(args[1]) * args[2] 
    for i in range(len(list_generated)): # check all blocks have the correct number of trials
        assert len(list_generated[i]) == correct_trial_num, "trials per block not correct"

    return list_generated

### Create the listgen function

In [3]:
def gen_pdm_list(levels_coh, interogation_times, reps, blocks,
                 trials=False, counterbalance_hard=True, rand_seed=False,
                 file_name=False,):
    """
    This returns a list of a list of dictionaries. Organized by block(list), trial(list), stimulus(dict)
    TODO: make sure trial nomenclature is correct and consistent
    Left and right coherence, and interogation time are all fully crossed. 
        This will result in a single rep being len(levels_coh)*len(levels_coh)*len(interogation_times) trials long.
        A block is made up of an interger number of reps
    If trials is specified, then counterbalancing is difficult
        If counterbalance_hard is true, then reps is rounded to an interger to get as close as possible to the requested trials
        If counterbalance_hard is not true, then I do the same thing because I couldn't decide what should get done
    """

    # Used if you want to specify the number of trials and not just the number of repetitions
    # (Becky you can skip this if you want)
    if trials:
        if counterbalance_hard:
            assert trials % (len(levels_coh) * len(levels_coh) * len(interogation_times)) == 0, 'Cannot maintain counterbalancing with current trials number'
            reps = np.round(trials / (len(levels_coh) * len(levels_coh) * len(interogation_times)), dtype=int)
        else:
            # TODO what to do if counterbalance is not hard?
            reps = np.round(trials / (len(levels_coh) * len(levels_coh) * len(interogation_times)), dtype=int)

    # Checks that the file name is a string and ends correctly
    # This could be done when saving, but rather do 1 if statement twice than run the whole thing and it break
    if file_name:
        assert type(file_name)==str, "file_name is not a string"
        # Check that the file name ends with something like .pickle
        if (not file_name.endswith('.pickle')) & (not file_name.endswith('.pkl')):
            file_name += '.pickle' 

    # Here is the main loop to generate all the possible conditions
    trial_base = []
    for left in levels_coh:
        for right in levels_coh:
            for inter in interogation_times:
                # This will result in a set of every combination of the three conditions
                trial_base.append({'left_coh': left,
                                   'right_coh': right,
                                   'inter_time': inter})
    
    # set seed if there is one to set
    if rand_seed: random.seed(rand_seed)

    # create the reps per block
    block_base = trial_base * reps
    all_blocks = []
    # for each block make copy and shuffle the 
    for i in range(blocks):
        # shuffle stuff
        loop_thing = block_base.copy()
        random.shuffle(loop_thing)
        all_blocks.append(loop_thing)

    # save out the list 
    if file_name:
        pickle.dump(all_blocks, open(file_name, 'wb'))

    # return the list too 'cause that is often convenient
    return all_blocks
    

### Run the test function 

In [4]:
# run a test case of the function with simple numbers
test_list = list_gen_test([.1,.2],[10,30],2,3)
# print it out to visually inspect the randomness
for i, block in enumerate(test_list):
    print(f"Block: {i}")
    print(block)

Block: 0
[{'left_coh': 0.2, 'right_coh': 0.2, 'inter_time': 10}, {'left_coh': 0.2, 'right_coh': 0.1, 'inter_time': 10}, {'left_coh': 0.2, 'right_coh': 0.1, 'inter_time': 30}, {'left_coh': 0.1, 'right_coh': 0.2, 'inter_time': 10}, {'left_coh': 0.1, 'right_coh': 0.2, 'inter_time': 30}, {'left_coh': 0.1, 'right_coh': 0.1, 'inter_time': 30}, {'left_coh': 0.1, 'right_coh': 0.1, 'inter_time': 10}, {'left_coh': 0.2, 'right_coh': 0.2, 'inter_time': 30}, {'left_coh': 0.1, 'right_coh': 0.1, 'inter_time': 10}, {'left_coh': 0.2, 'right_coh': 0.2, 'inter_time': 10}, {'left_coh': 0.1, 'right_coh': 0.1, 'inter_time': 30}, {'left_coh': 0.2, 'right_coh': 0.1, 'inter_time': 10}, {'left_coh': 0.2, 'right_coh': 0.1, 'inter_time': 30}, {'left_coh': 0.1, 'right_coh': 0.2, 'inter_time': 10}, {'left_coh': 0.2, 'right_coh': 0.2, 'inter_time': 30}, {'left_coh': 0.1, 'right_coh': 0.2, 'inter_time': 30}]
Block: 1
[{'left_coh': 0.2, 'right_coh': 0.2, 'inter_time': 10}, {'left_coh': 0.1, 'right_coh': 0.2, 'inter_ti

## Final List generated

In [5]:
levels_coh = [0.0, 0.1, 0.2, 0.3]
interogation_times = [.200, .300, .500, .800]

reps = 3
blocks = 5

final_list = gen_pdm_list(levels_coh=levels_coh, interogation_times=interogation_times, reps=reps, blocks=blocks,
                          file_name="A01_pdm_list_csh4hf")


In [6]:
# Print final list
for i, block in enumerate(final_list):
    print(f"Block: {i}")
    for j, trial in enumerate(block):
        print(f"\tTrial number {j}: {trial}")

Block: 0
	Trial number 0: {'left_coh': 0.1, 'right_coh': 0.1, 'inter_time': 0.3}
	Trial number 1: {'left_coh': 0.1, 'right_coh': 0.2, 'inter_time': 0.2}
	Trial number 2: {'left_coh': 0.2, 'right_coh': 0.2, 'inter_time': 0.8}
	Trial number 3: {'left_coh': 0.0, 'right_coh': 0.0, 'inter_time': 0.2}
	Trial number 4: {'left_coh': 0.2, 'right_coh': 0.0, 'inter_time': 0.2}
	Trial number 5: {'left_coh': 0.2, 'right_coh': 0.3, 'inter_time': 0.2}
	Trial number 6: {'left_coh': 0.1, 'right_coh': 0.3, 'inter_time': 0.5}
	Trial number 7: {'left_coh': 0.0, 'right_coh': 0.1, 'inter_time': 0.2}
	Trial number 8: {'left_coh': 0.1, 'right_coh': 0.3, 'inter_time': 0.3}
	Trial number 9: {'left_coh': 0.1, 'right_coh': 0.3, 'inter_time': 0.3}
	Trial number 10: {'left_coh': 0.0, 'right_coh': 0.3, 'inter_time': 0.3}
	Trial number 11: {'left_coh': 0.1, 'right_coh': 0.2, 'inter_time': 0.8}
	Trial number 12: {'left_coh': 0.2, 'right_coh': 0.1, 'inter_time': 0.3}
	Trial number 13: {'left_coh': 0.0, 'right_coh': 0.1